# Faster R-CNN MobileNetV3 Large


In [162]:
import torch
import numpy as np
import pandas as pd
import stuff
import os
import cv2

from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_fpn
from torchvision.transforms import functional as F
from torchvision.transforms import Compose, ToTensor, Resize
from torchsummary import summary
from torch.utils.data.dataset import random_split

from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [163]:
IMAGE_FOLDER = stuff.IMAGES_FOLDER
BALL_CSV = stuff.BALL_CSV
LINE_CSV = stuff.LINE_CSV

## Dataset


In [183]:
class CustomDataset(Dataset):
    def __init__(self, images, ball_df, line_df, transform=None):
        self.images = images
        self.ball_df = ball_df
        self.line_df = line_df

        self.boxes, self.labels, self.imgs = self.organise_data(
            images, ball_df, line_df
        )
        self.transform = transform

    def __getitem__(self, idx):
        image = self.transform(np.array(self.imgs[idx]), dtype=torch.float32)
        box = self.transform(self.boxes[idx], dtype=torch.int64)
        label = self.transform(self.labels[idx], dtype=torch.int64)

        return image, box, label

    def __len__(self):
        return len(self.images)

    def organise_data(self, images, ball_df, line_df):
        imgs = []
        boxes = []
        labels = []
        for i, img in enumerate(images):
            ball_topleft = [ball_df.iloc[i]["x"], ball_df.iloc[i]["y"]]
            ball_bottomright = [
                ball_df.iloc[i]["x"] + (ball_df.iloc[i]["width"]),
                ball_df.iloc[i]["y"] + (ball_df.iloc[i]["height"]),
            ]
            ball_data = [ball_topleft, ball_bottomright]

            line_topleft = [line_df.iloc[i]["x"], line_df.iloc[i]["y"]]
            line_bottomright = [
                line_df.iloc[i]["x"] + (line_df.iloc[i]["width"]),
                line_df.iloc[i]["y"] + (line_df.iloc[i]["height"]),
            ]
            line_data = [line_topleft, line_bottomright]

            boxes.append([ball_data, line_data])
            labels.append([1, 2])

            new_img = np.transpose(img, (2, 0, 1))
            imgs.append(new_img)

        return boxes, labels, imgs

In [184]:
def get_image_files(IMAGE_FOLDER, IMAGE_FILE_TYPE):
    image_files = []
    for file in os.listdir(IMAGE_FOLDER):
        if file.endswith(IMAGE_FILE_TYPE):
            image_files.append(file)
    return image_files


def load_image_files(image_files, IMAGE_FOLDER):
    images = []
    for file in image_files:
        image = cv2.imread(IMAGE_FOLDER + "/" + file)
        images.append(image)
    return images

In [185]:
image_files = get_image_files(IMAGE_FOLDER, "jpg")
images = load_image_files(image_files, IMAGE_FOLDER)
np.array(images).shape

(21, 280, 504, 3)

In [186]:
ball_df = pd.read_csv(BALL_CSV)
line_df = pd.read_csv(LINE_CSV)

In [187]:
dataset = CustomDataset(images, ball_df, line_df, torch.tensor)

In [ ]:
train_inds, val_inds,

## Dataloader


In [188]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

## Training


In [203]:
model = fasterrcnn_mobilenet_v3_large_fpn(pretrained=False)
input_size = dataset[0][0].shape
input_size

c:\Users\loicl\Documents\Enlightenment\Programming\Artificial Intelligence\Object Detection\Over-the-line--Football-line-detector\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to C:\Users\loicl/.cache\torch\hub\checkpoints\mobilenet_v3_large-8738ca79.pth
100.0%


torch.Size([3, 280, 504])